In [1]:
# from utility.MoleculeParser import MoleculeData
# from utility.QMUQUBO import QMUQUBO
# from utility.AnnealerOptimizer import Annealer
# from utility.ResultProcess import ResultParser
# import time

# timestamp = time.strftime("%Y%m%d-%H")

# import packages, including those to connect with AWS BraKet

import numpy as np
import pandas as pd
import math
import os
import glob

from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

# Step 1: Prepare Data

In [2]:
# function to read in .ct file and give a list of known structure stems:

def actual_stems(seq_ss, seq_ps):
    
    with open(subdirectory+"/"+seq_ss) as file:
        lines = file.readlines()
    
    with open(subdirectory+"/"+seq_ps) as file:
        fasta_lines = file.readlines()
    
    rna = fasta_lines[1]
    
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line

    for i in range(0, len(lines)):
        line = lines[i].strip().split()
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
        if ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 0
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl = 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl = 2
        
        last_line = line
        
    return stems_actual

In [3]:
# function to generate list of potential stem pairs that form pseudoknots:

def potential_pseudoknots(stems_potential, pkp):

    pseudoknots_potential = []
    pseudoknot_penalty = pkp

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i,j,1]
    
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
        
                pseudoknot[2] = pseudoknot_penalty
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [4]:
# function to generate list of stem pairs that overlap:

def potential_overlaps(stems_potential):
    
    overlaps_potential = []
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            overlap = [i, j, 0]
    
            stem1_cspan1 = set(range(stem1[1]-int(stem1[2])+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-int(stem2[2])+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+int(stem1[2])))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+int(stem2[2])))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                overlap[2] = overlap_penalty
        
            overlaps_potential.append(overlap)
            
    return overlaps_potential

In [5]:
# function to read in .fasta file and generate list of potential stems at least 3 base-pairs long:

def potential_stems(seq_ps):
    
    with open(subdirectory+"/"+seq_ps) as file:
        lines = file.readlines()
    
    rna = lines[1]
    
    matrix = np.zeros((len(rna),len(rna)))
    for diag in range(0, len(matrix)):
        for row in range(0, len(matrix)-diag):
            col = row + diag
            base1 = rna[row]
            base2 = rna[col]
            if row != col:
                if ((base1 == ("A" or "a")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("A" or "a"))) or ((base1 == ("G" or "g")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 2
                if ((base1 == ("G" or "g")) and (base2 == ("C" or "c"))) or ((base1 == ("C" or "c")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 3
    
    stems_potential = []
    mu = 0

    for row in range(0, len(matrix)):
        for col in range (row, len(matrix)):
            if row != col:
                if matrix[row][col] != 0:
                    temp_row = row
                    temp_col = col
                    stem = [row+1,col+1,0]
                    length_N = 0
                    length_H = 0
                    while (matrix[temp_row][temp_col] != 0) and (temp_row != temp_col):
                        length_N+=1
                        length_H+=matrix[temp_row][temp_col]
                        temp_row+=1
                        temp_col-=1
                        if length_N >= 3:
                            stem[2] = int(length_H)
                            stems_potential.append(stem.copy())
                    if length_H > mu:
                        mu = length_H
    
    return [stems_potential, mu, rna, len(rna)]

In [6]:
# pk = ["wPKs", "woutPKs"]
pk = ["wPKs"]
# s  = ["s", "m", "l"]
s  = ["s"]
p  = [-1.0, -0.5, 0.0, 0.5, 1.0]
pl = ["n1", "np5", "0", "pp5", "p1"]

data = []

# Step 2: Build Model

In [7]:
# function to evaluate the energy of the known structure under the model Hamiltonian:

def energy(stems_actual, pkp):
    
    cl = 1
    cb = 1
    k = 0
    
    pseudoknots_actual = potential_pseudoknots(stems_actual, pkp)
    cost = 0
    mu = max(list(map(list, zip(*stems_actual)))[2])
    
    for i in range(0, len(stems_actual)):
        cost += cl*((stems_actual[i][2]**2)-2*mu*stems_actual[i][2]+mu**2)-cb*(stems_actual[i][2]**2)
        for j in range(i+1, len(stems_actual)):
            cost -= 2*cb*stems_actual[i][2]*stems_actual[j][2]*pseudoknots_actual[k][2]
            k += 1
    
    return cost

In [8]:
# function to generate the Hamiltonian of a given RNA structure from potential stems, overlaps, and pseudoknots:

def model(stems_potential, pseudoknots_potential, overlaps_potential, mu):
    
    L = {}
    Q = {}
    cl = 1
    cb = 1
    k = 0

    for i in range(0, len(stems_potential)):
        L[str(i)] = cl*((stems_potential[i][2]**2)-2*mu*stems_potential[i][2]+mu**2)-cb*(stems_potential[i][2]**2)
        for j in range(i+1, len(stems_potential)):
            Q[(str(i), str(j))] = -2*cb*stems_potential[i][2]*stems_potential[j][2]*pseudoknots_potential[k][2]+overlaps_potential[k][2]
            k += 1
    
    return L, Q

# Step 3: Optimize Configuration

In [9]:
for a in range(0, len(pk)):
    for b in range(0, len(s)):
        for c in range(0, len(pl)):
            
            subdirectory = './known_structures/'+pk[a]+'/'+s[b]

            ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
            fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

            bprna_id = []
            size = []
            pks = []
            pk_penalty = []

            penalty = pl[c]

            for i in range(0, len(ct)):
                bprna_id.append(ct[i].split('.')[0])
                size.append(subdirectory.split("/")[3])
                if subdirectory.split("/")[2] == "wPKs":
                    pks.append("T")
                else:
                    pks.append("F")
                pk_penalty.append(penalty)
            
            stems_a    = []
            energies_a = []
            stems_p    = []
            pks_p      = []
            ols_p      = []
            models     = []

            for index in range(0, len(ct)):
                stems_a.append(actual_stems(ct[index], fasta[index]))
                energies_a.append(energy(stems_a[index], p[c]))
                stems_p.append(potential_stems(fasta[index]))
                pks_p.append(potential_pseudoknots(stems_p[index][0], p[c]))
                ols_p.append(potential_overlaps(stems_p[index][0]))
                models.append(model(stems_p[index][0], pks_p[index], ols_p[index], stems_p[index][1]))
                                
            print(models)
            problem = []
            # print(pk_penalty)

[({'0': 28.0, '1': 0.0, '2': -84.0, '3': -28.0, '4': 28.0, '5': -28.0, '6': 28.0, '7': -28.0, '8': -84.0, '9': -140.0, '10': -196.0, '11': 28.0, '12': 28.0, '13': 28.0, '14': -28.0, '15': 28.0, '16': -28.0, '17': -84.0, '18': 28.0, '19': -28.0, '20': -84.0, '21': -140.0, '22': 28.0, '23': 28.0, '24': 28.0, '25': -28.0, '26': 28.0, '27': -28.0, '28': -84.0, '29': 28.0, '30': -28.0, '31': -84.0, '32': -140.0, '33': 28.0, '34': 28.0, '35': 28.0, '36': -28.0, '37': 28.0, '38': -28.0, '39': -84.0, '40': 28.0, '41': -28.0, '42': -84.0, '43': -168.0, '44': 28.0, '45': 28.0, '46': 28.0, '47': -28.0, '48': 28.0, '49': -28.0, '50': -112.0, '51': 28.0, '52': -28.0, '53': 28.0, '54': 28.0, '55': 28.0, '56': -56.0, '57': 28.0, '58': 28.0, '59': 0.0, '60': 0.0, '61': -28.0, '62': -56.0, '63': -112.0, '64': -28.0, '65': -28.0, '66': -28.0, '67': -28.0, '68': 0.0, '69': -56.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0}, {('0', '1'): 999916.0, ('0', '2'): 999880.0, ('0', '3'): 1000096.0, ('0', '4'): 1

# Step 4: PostProcess Result